In [ ]:
import cv2
from super_gradients.common.object_names import Models
from super_gradients.training import models
import numpy as np
import urllib

def open_input(input_path):
    """
    Abre a entrada com base no tipo (imagem, vídeo ou streaming) e retorna
    o objeto VideoCapture correspondente.
    """
    if input_path.startswith('http'):  # Se for um link de streaming
        stream = urllib.request.urlopen(input_path)
        bytes = bytearray()
        while True:
            bytes += stream.read(1024)
            a = bytes.find(b'\xff\xd8')
            b = bytes.find(b'\xff\xd9')
            if a != -1 and b != -1:
                jpg = bytes[a:b + 2]
                bytes = bytes[b + 2:]
                frame = cv2.imdecode(np.frombuffer(jpg, dtype=np.uint8), cv2.IMREAD_COLOR)
                return cv2.VideoCapture(frame)
    elif input_path.endswith(('jpg', 'jpeg', 'png', 'bmp')):  # Se for uma imagem
        return cv2.VideoCapture(input_path)
    else:  # Se for um vídeo
        return cv2.VideoCapture(input_path)

def process_input(input_path):
    """
    Processa a entrada com base no tipo (imagem, vídeo ou streaming) e retorna
    o objeto VideoCapture correspondente e a largura e altura dos quadros.
    """
    cap = open_input(input_path)

    # Obtém a largura e a altura do quadro do vídeo
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    return cap, frame_width, frame_height

def process_frame(frame, input_type):
    """
    Processa um único quadro do vídeo.
    Retorna o quadro com as previsões desenhadas
    e uma lista de todas as previsões.
    """
    # Faz previsões no quadro
    predictions = model.predict(frame, conf=confidence_threshold)

    # Extrai informações das previsões apenas para a classe de pessoa
    frame_predictions = []
    for prediction in predictions:
        class_names = prediction.class_names
        labels = prediction.prediction.labels
        confidence = prediction.prediction.confidence
        bboxes = prediction.prediction.bboxes_xyxy

        for label, conf, bbox in zip(labels, confidence, bboxes):
            # Verifica se o rótulo é para uma pessoa
            if class_names[int(label)] == 'person' and conf >= confidence_threshold:
                # Salva a previsão no formato desejado
                frame_predictions.append({
                    "class_name": class_names[int(label)],
                    "confidence": conf,
                    "bbox": bbox
                })

                # Desenha a caixa delimitadora e o rótulo no quadro
                xmin, ymin, xmax, ymax = map(int, bbox)
                cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
                cv2.putText(frame, f'Person: {conf:.2f}', (xmin, ymin - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    return frame, frame_predictions

# Define o caminho da entrada (imagem, vídeo ou link de streaming)
INPUT_PATH = "/kaggle/input/video/videoplayback (1).mp4"

# Processa a entrada com base no tipo
cap, frame_width, frame_height = process_input(INPUT_PATH)

# Define o codec e cria o objeto VideoWriter
output_path = "/kaggle/working/output_video.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, 20.0, (frame_width, frame_height))

# Carrega o modelo YOLO-NAS-L
model = models.get(Models.YOLO_NAS_L, pretrained_weights="coco")

# Define o limite de confiança inicial
confidence_threshold = 0.2

# Cria um arquivo de log para salvar as previsões
log_file = open("/kaggle/working/predictions_log.txt", "w")

# Processa cada frame do vídeo
frame_count = 0
while cap.isOpened() and frame_count < 1000:
    ret, frame = cap.read()
    if not ret:
        break

    # Processa o quadro e obtém as previsões
    frame_processed, frame_predictions = process_frame(frame, input_type='video')

    # Escreve o quadro processado no vídeo de saída
    out.write(frame_processed)

    # Salva as previsões no arquivo de log
    for prediction in frame_predictions:
        log_file.write(f"Class: {prediction['class_name']}, Confidence: {prediction['confidence']}, BBox: {prediction['bbox']}\n")

    frame_count += 1

# Fecha o arquivo de log
log_file.close()

# Libera os objetos de captura de vídeo e gravação
cap.release()
out.release()

print("Vídeo com previsões salvo com sucesso em:", output_path)
print("Arquivo de log com as previsões salvo com sucesso.")


[2024-05-11 13:37:53] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.
[2024-05-11 13:37:53] INFO - checkpoint_utils.py - Successfully loaded pretrained weights for architecture yolo_nas_l
[2024-05-11 13:37:53] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-05-11 13:37:54] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-05-11 13:37:55] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-05-11 13:37:56] INFO - pipelines.py - Fusing some of the mod